In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray,netCDF4

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.5
numpy 1.11.1
xarray 0.9.1
netCDF4 1.2.4
Thu Aug 24 10:26:10 2017 CEST

In [2]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")

In [3]:
## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd 
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar

%matplotlib inline

In [4]:
## datasets

#NATL60

dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/"
file="NATL60-CJM165_y2012m06d14.1d_gridT.nc"

tfile=dir+file

!ls $tfile


/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2012m06d14.1d_gridT.nc


In [6]:
dsN = xr.open_dataset(tfile, chunks={'deptht':30 ,'time_counter':1})

In [15]:
dsN

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, deptht: 300, time_counter: 1, x: 5422, y: 3454)
Coordinates:
    nav_lat               (y, x) float32 26.5648 26.5648 26.5648 26.5648 ...
    nav_lon               (y, x) float32 -81.4512 -81.4346 -81.4179 -81.4012 ...
  * deptht                (deptht) float32 0.480455 1.55879 2.79421 4.18731 ...
    time_centered         (time_counter) datetime64[ns] 2012-06-14T12:00:00
  * time_counter          (time_counter) datetime64[ns] 2012-06-14T12:00:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    deptht_bounds         (deptht, axis_nbounds) float32 0.0 1.00003 ...
    votemper              (time_counter, deptht, y, x) float64 nan nan nan ...
    time_centered_bounds  (time_counter, axis_nbounds) float64 1.718e+09 ...
    time_counter_bounds   (time_counter, axis_nbounds) float64 1.718e+09 ...
    vosaline              (time_counter, deptht, y, x) float64 nan nan nan ...
    sossheig              (time_coun

In [7]:
latN = dsN.nav_lat
lonN = dsN.nav_lon
tempN=dsN.votemper
saltN=dsN.vosaline
depN = dsN.deptht
timN = dsN.time_counter

In [8]:
observation_lon=-63.784
observation_lat=37.495

In [50]:
#decoupage grossier autour de la position du profile ARGO pour un niveau vertical

indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_tempBOX=tempN[:,0,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_saltBOX=saltN[:,0,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

model_lonBOX_array=model_lonBOX.values
model_latBOX_array=model_latBOX.values
model_tempBOX_array=model_tempBOX.values
model_saltBOX_array=model_saltBOX.values
time_array=timN.values
t_dim=np.arange(1)
x_dim=np.arange(122)
y_dim=np.arange(159)


In [51]:
model_lonBOX_array

array([[-64.77238464, -64.75571442, -64.73905182, ..., -62.78866196,
        -62.77199173, -62.7553215 ],
       [-64.77262878, -64.75595856, -64.73929596, ..., -62.7889061 ,
        -62.77223587, -62.75556183],
       [-64.77287292, -64.75621033, -64.7395401 , ..., -62.78915024,
        -62.77248001, -62.75580978],
       ..., 
       [-64.81837463, -64.80170441, -64.78503418, ..., -62.83447266,
        -62.81779861, -62.80112457],
       [-64.81871796, -64.80205536, -64.78538513, ..., -62.83482361,
        -62.81814575, -62.80147171],
       [-64.81906891, -64.80239868, -64.78573608, ..., -62.83517075,
        -62.8184967 , -62.80182266]], dtype=float32)

In [52]:
print time_array.shape
print model_latBOX_array.shape
print model_lonBOX_array.shape
print model_tempBOX_array.shape
print model_saltBOX_array.shape
print t_dim.shape
print x_dim.shape
print y_dim.shape

(1,)
(159, 122)
(159, 122)
(1, 159, 122)
(1, 159, 122)
(1,)
(122,)
(159,)


In [53]:

d = {}
d['time_counter'] = ('time_counter',t_dim)
d['y'] = ('y',y_dim)
d['x'] = ('x',x_dim)
d['nav_lat'] = (['y','x'],model_latBOX_array)
d['nav_lon'] = (['y','x'],model_lonBOX_array)

d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)

dset = xr.Dataset(d)
dset

<xarray.Dataset>
Dimensions:       (time_counter: 1, x: 122, y: 159)
Coordinates:
  * time_counter  (time_counter) int64 0
  * y             (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * x             (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Data variables:
    votemper      (time_counter, y, x) float64 24.43 24.44 24.44 24.45 24.45 ...
    nav_lat       (y, x) float32 36.4767 36.477 36.4772 36.4774 36.4776 ...
    nav_lon       (y, x) float32 -64.7724 -64.7557 -64.7391 -64.7224 ...
    vosaline      (time_counter, y, x) float64 36.36 36.36 36.36 36.36 36.36 ...

In [54]:
latB = dset.nav_lat
lonB = dset.nav_lon
model_temperatureB = dset.votemper
model_salinityB = dset.vosaline


In [55]:
model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
lon_stacked = lonB.stack(profile=('x', 'y'))
lat_stacked = latB.stack(profile=('x', 'y'))


In [56]:
distance_threshold = 0.25
square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
is_close_to_observation = square_distance_to_observation < distance_threshold**2

model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

In [58]:
model_mean = model_temperature_near_observation.mean()
    
percentile_10= model_temperature_near_observation.quantile(0.1)
    
percentile_90= model_temperature_near_observation.quantile(0.9)


In [64]:
model_mean

<xarray.DataArray ()>
array(24.728003585043457)

In [67]:
#loop over vertical level
def profile_mean_percent(k):

    #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_tempBOX=tempN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_saltBOX=saltN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    model_tempBOX_array=model_tempBOX.values
    model_saltBOX_array=model_saltBOX.values
    time_array=timN.values
    t_dim=np.arange(1)
    x_dim=np.arange(122)
    y_dim=np.arange(159)

    # construction d'un nouveau xarray
    d = {}
    d['time_counter'] = ('time_counter',t_dim)
    d['y'] = ('y',y_dim)
    d['x'] = ('x',x_dim)
    d['nav_lat'] = (['y','x'],model_latBOX_array)
    d['nav_lon'] = (['y','x'],model_lonBOX_array)
    
    d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
    d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    
    dset = xr.Dataset(d)
    
    latB = dset.nav_lat
    lonB = dset.nav_lon
    model_temperatureB = dset.votemper
    model_salinityB = dset.vosaline
    
    # selection plus fine des profils
    
    model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
    model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
    lon_stacked = lonB.stack(profile=('x', 'y'))
    lat_stacked = latB.stack(profile=('x', 'y'))
    
    distance_threshold = 0.25
    square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
    is_close_to_observation = square_distance_to_observation < distance_threshold**2
    
    model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
    model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
    lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
    lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)
    
    model_mean = model_temperature_near_observation.mean()
    percentile_10= model_temperature_near_observation.quantile(0.1)
    percentile_90= model_temperature_near_observation.quantile(0.9)

    return model_mean,percentile_10,percentile_90

In [68]:
%timeit profile_mean_percent(0)

1 loop, best of 3: 26.4 s per loop


In [72]:
%timeit profile_mean_percent(0)

1 loop, best of 3: 25.9 s per loop


In [69]:
#loop over vertical level
def profile_mean(k):

    #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_tempBOX=tempN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_saltBOX=saltN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    model_tempBOX_array=model_tempBOX.values
    model_saltBOX_array=model_saltBOX.values
    time_array=timN.values
    t_dim=np.arange(1)
    x_dim=np.arange(122)
    y_dim=np.arange(159)

    # construction d'un nouveau xarray
    d = {}
    d['time_counter'] = ('time_counter',t_dim)
    d['y'] = ('y',y_dim)
    d['x'] = ('x',x_dim)
    d['nav_lat'] = (['y','x'],model_latBOX_array)
    d['nav_lon'] = (['y','x'],model_lonBOX_array)
    
    d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
    d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    
    dset = xr.Dataset(d)
    
    latB = dset.nav_lat
    lonB = dset.nav_lon
    model_temperatureB = dset.votemper
    model_salinityB = dset.vosaline
    
    # selection plus fine des profils
    
    model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
    model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
    lon_stacked = lonB.stack(profile=('x', 'y'))
    lat_stacked = latB.stack(profile=('x', 'y'))
    
    distance_threshold = 0.25
    square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
    is_close_to_observation = square_distance_to_observation < distance_threshold**2
    
    model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
    model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
    lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
    lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)
    
    model_mean = model_temperature_near_observation.mean()

    return model_mean

In [70]:
%timeit profile_mean(0)

1 loop, best of 3: 26.7 s per loop


In [71]:
%timeit profile_mean(0)

1 loop, best of 3: 25.9 s per loop


In [73]:
%timeit profile_mean(1)

1 loop, best of 3: 24.7 s per loop
